<a href="https://colab.research.google.com/github/rahulsm27/ML/blob/main/Newton_Method_for_Neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn,optim
import torch.nn.functional as F
import pandas as pd
import numpy as np

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"


In [313]:
# Creating our own ConvNet
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)

        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 1000)
        self.fc2 = nn.Linear(1000, 10)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.dropout(x)
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout(x)

        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = ConvNet().to(device)
torch.nn.init.xavier_uniform_(model.conv1.weight)
torch.nn.init.xavier_uniform_(model.conv2.weight)


Parameter containing:
tensor([[[[-0.0562,  0.0806, -0.0634,  0.0177, -0.0316],
          [-0.0435,  0.0756, -0.0143,  0.0510,  0.0132],
          [-0.0791, -0.0168, -0.0100, -0.0752,  0.0842],
          [ 0.0942, -0.0933,  0.0033,  0.0166, -0.0523],
          [-0.0627,  0.0404,  0.0753, -0.0465, -0.0319]],

         [[ 0.0064,  0.0204, -0.0382,  0.0957,  0.0203],
          [ 0.0852,  0.0372,  0.0685, -0.0936, -0.0037],
          [ 0.0002, -0.0031, -0.0651, -0.0990, -0.0370],
          [-0.0496, -0.0905,  0.0451,  0.0843, -0.0990],
          [-0.0979, -0.0839,  0.0102, -0.0869, -0.0248]],

         [[-0.0057, -0.0373, -0.0339, -0.0249,  0.0987],
          [ 0.0215, -0.0191, -0.0602,  0.0077,  0.0968],
          [ 0.0145,  0.0719, -0.0631, -0.0027, -0.0089],
          [ 0.0338,  0.0720,  0.0027, -0.0958, -0.0997],
          [-0.0817,  0.0081,  0.0611, -0.0911, -0.0872]],

         [[-0.0150,  0.0743,  0.0557,  0.0465,  0.0227],
          [-0.0504,  0.0464,  0.0592,  0.0701, -0.0417],
   

In [314]:
from torchvision import transforms
from torchvision.datasets import CIFAR10
train_transforms = transforms.Compose([ transforms.ToTensor()])
dataset = CIFAR10(root="./test/", train=False, download=True, transform=train_transforms)


train_set, test_set = torch.utils.data.random_split(dataset, [8000, 2000])

trainloader = torch.utils.data.DataLoader( train_set, batch_size=1024,shuffle=True)
testloader = torch.utils.data.DataLoader( test_set,batch_size=1024, shuffle=True)

Files already downloaded and verified


In [315]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.LBFGS(model.parameters(),lr=0.35,max_iter=4,history_size=10,line_search_fn='strong_wolfe')



In [316]:
N_EPOCHS = 50
for epoch in range(N_EPOCHS):
  epoch_loss = 0.0
  i = 0
  model.train()
  for inputs, labels in trainloader:
    inputs = inputs.to(device)
    labels = labels.to(device)

    def closure():
      optimizer.zero_grad()
      outputs = model(inputs)
      loss = criterion(outputs, labels)
      loss.backward(retain_graph=True)

      return loss

    optimizer.step(closure)

    outputs = model(inputs)
    loss = criterion(outputs, labels)
    epoch_loss += loss.item()

  val_loss = 0.0
  model.eval()
  for inputs, labels in testloader:
    inputs = inputs.to(device)
    labels = labels.to(device)

    outputs = model(inputs)
    loss = criterion(outputs, labels)
    val_loss += loss.item()
  print("Epoch: {} Train Loss: {} Val Loss: {} ".format(epoch, epoch_loss/len(trainloader), val_loss/len(testloader)))



Epoch: 0 Train Loss: 2.302582234144211 Val Loss: 2.302242159843445 
Epoch: 1 Train Loss: 2.302254170179367 Val Loss: 2.302234172821045 
Epoch: 2 Train Loss: 2.301358014345169 Val Loss: 2.3020946979522705 
Epoch: 3 Train Loss: 2.302258551120758 Val Loss: 2.3020176887512207 
Epoch: 4 Train Loss: 2.300609827041626 Val Loss: 2.298862934112549 
Epoch: 5 Train Loss: 2.199757754802704 Val Loss: 2.0873111486434937 
Epoch: 6 Train Loss: 2.050920605659485 Val Loss: 2.0390113592147827 
Epoch: 7 Train Loss: 2.0329954624176025 Val Loss: 2.0269094705581665 
Epoch: 8 Train Loss: 2.0245636105537415 Val Loss: 2.0117079615592957 
Epoch: 9 Train Loss: 2.013158768415451 Val Loss: 2.009145140647888 
Epoch: 10 Train Loss: 1.9904993027448654 Val Loss: 2.0010342597961426 
Epoch: 11 Train Loss: 1.9889756739139557 Val Loss: 1.9964798092842102 
Epoch: 12 Train Loss: 1.9787068963050842 Val Loss: 1.98788321018219 
Epoch: 13 Train Loss: 1.950659066438675 Val Loss: 1.964683711528778 
Epoch: 14 Train Loss: 1.91693474

In [320]:
y_pred = model(test_set[901][0].unsqueeze(0).to(device))
print(f" Model predicted label : {np.argmax(y_pred.detach().cpu().numpy())}")
print(f" Actual label : {test_set[901][1]}")

 Model predicted label : 1
 Actual label : 1


In [318]:
y_pred = model(train_set[9][0].unsqueeze(0).to(device))
print(f" Model predicted label : {np.argmax(y_pred.detach().cpu().numpy())}")
print(f" Actual label : {train_set[9][1]}")

 Model predicted label : 1
 Actual label : 1
